In [34]:
import os
import sys
import re
# Get the absolute path to the parent directory (assumes this file is in 'condensation')
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)
# autoreload modules
%load_ext autoreload
%autoreload 2
from typing import List, Dict
import datetime
from dataclasses import dataclass
import pandas as pd
from chatbot_api.providers.openai import OpenAIProvider
from chatbot_api.base import Role, Message

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
class BatchAnalysisParser:
    def __init__(self, analysis_text: str):
        self.analysis_text = analysis_text

    def parse_arguments(self) -> List[Dict[str, str]]:
        """
        Parses the individual arguments' analysis section.
        Returns a list of dictionaries where each dictionary represents a single argument.
        """
        argument_pattern = re.compile(
            r"<ANALYYSI>\s*ARGUMENTTI (\d+):\s*"
            r"Jäljitettävyys:\s*\[(.*?)\]\s*"
            r"Lisäykset:\s*\[(.*?)\]\s*"
            r"Poistot:\s*\[(.*?)\]\s*"
            r"Muutokset:\s*\[(.*?)\]\s*"
            r"Relevanttius:\s*\[(.*?)\]\s*"
            r"Perustelut:\s*\[(.*?)\]\s*</ANALYYSI>",
            re.DOTALL
        )
        matches = argument_pattern.finditer(self.analysis_text)
        arguments = []

        for match in matches:
            arguments.append({
                "index": match.group(1),
                "jaljitettavyys": match.group(2).strip(),
                "lisaykset": match.group(3).strip(),
                "poistot": match.group(4).strip(),
                "muutokset": match.group(5).strip(),
                "relevanttius": match.group(6).strip(),
                "perustelut": match.group(7).strip(),
            })
        
        return arguments

    def parse_summary(self) -> Dict[str, str]:
        """
        Parses the summary section of the output.
        Returns a dictionary containing the summary information.
        """
        summary_pattern = re.compile(
            r"<YHTEENVETO>\s*"
            r"Yhteenveto kaikista tapahtuneista muutoksista:\s*\[(.*?)\]\s*"
            r"Muutosten perustelut:\s*\[(.*?)\]\s*"
            r"Kehityskohteet:\s*\[(.*?)\]\s*</YHTEENVETO>",
            re.DOTALL
        )
        match = summary_pattern.search(self.analysis_text)
        
        if match:
            return {
                "yhteenveto": match.group(1).strip(),
                "perustelut": match.group(2).strip(),
                "kehityskohteet": match.group(3).strip(),
            }
        return {}

    def parse(self) -> Dict[str, List[Dict[str, str]]]:
        """
        Combines the parsed arguments and summary into a single structure.
        """
        return {
            "arguments": self.parse_arguments(),
            "summary": self.parse_summary(),
        }

In [36]:
@dataclass
class Argument:
    """
    A simplified argument class that tracks the main point and its history.
    The history allows us to see how the argument evolved over iterations.
    """
    main_argument: str
    argument_history: List[str]  # Tracks how this argument has changed over time

    def update(self, new_arg: str):
        """Record a new version of this argument."""
        self.argument_history.append(self.main_argument)
        self.main_argument = new_arg

class ArgumentSynthesizer:
    def __init__(self, llm_provider, batch_size: int = 10):
        self.llm_provider = llm_provider
        self.batch_size = batch_size
        self.current_arguments: List[Argument] = []
        
        # Template for initial argument creation, now in Finnish with more flexible argument count
        self.INITIAL_TEMPLATE = """
        ### Ohjeet:
        1. Käy läpi seuraavat kommentit aiheesta: "{topic}"
        2. Muodosta 2-4 pääargumenttia, jotka kuvaavat keskeisiä näkökulmia:
           - Luo argumentteja vain sen verran kuin on tarpeellista erilaisten näkökulmien esittämiseksi
           - Jokaisen argumentin tulisi edustaa selkeästi erottuvaa näkökantaa
           - Argumenttien tulisi olla tarpeeksi laajoja kattaakseen toisiinsa liittyvät ajatukset
           - Varmista, että argumentit on kirjoitettu selkeästi ja kattavasti
        3. Huomioi erityisesti:
           - Argumenttien määrä riippuu kommenttien sisällön monipuolisuudesta
           - On parempi luoda vähemmän laadukkaita argumentteja kuin useita pinnallisia

        ### Analysoitavat kommentit:
        {comments_text}

        ### Tulostusmuoto:
        <ARGUMENTS>
        ARGUMENTTI: [Ensimmäinen kattava argumentti]
        ARGUMENTTI: [Toinen kattava argumentti]
        </ARGUMENTS>
        """
        
        # Template for synthesis, also in Finnish
        self.SYNTHESIS_TEMPLATE = """
        ### Ohjeet:
        1. Käy läpi nykyiset argumentit ja uudet kommentit aiheesta: "{topic}"
        2. Kehitä ja täydennä argumenttirakennetta:
           - Yhdistä argumentit, jotka edustavat läheisesti toisiinsa liittyviä näkökulmia
           - Jaa argumentit, jos ne sisältävät selvästi erillisiä näkökantoja
           - Luo uusia argumentteja, jos kommentit paljastavat käsittelemättömiä näkökulmia
           - Muokkaa olemassa olevia argumentteja kuvaamaan paremmin keskustelun koko laajuutta
           - Poista tai korvaa argumentit, jotka eivät enää tehokkaasti edusta keskeisiä näkökulmia
        3. Tärkeää:
           - Argumenttien lopullinen määrä määräytyy sisällön perusteella
           - Keskity laatuun määrän sijasta
           - Jokaisen argumentin tulee olla kattava mutta selkeästi erillinen muista

        ### Nykyiset argumentit:
        {existing_arguments}

        ### Uudet kommentit:
        {new_comments}

        ### Tulostusmuoto:
        <ARGUMENTS>
        ARGUMENTTI: [Ensimmäinen päivitetty argumentti]
        ARGUMENTTI: [Toinen päivitetty argumentti]
        </ARGUMENTS>
        """

        self.BATCH_METRIC_TEMPLATE = """
            ### Ohjeet:
            1. Käy läpi seuraavat kommentit ja vanhat sekä uudet argumentit aiheesta: "{topic}".
            2. Arvioi uudet argumentit yksitellen seuraavien kohtien mukaan:
            - Jäljitettävyys: Kirjaa, mitkä kommentit vaikuttivat kunkin muuttuneen argumentin muutoksiin. Tai jos uskot argumentin olevan täysin uusi, kirjaa se.
            - Muutokset: Ilmoita, mitä lisäyksiä, poistamisia tai muutoksia argumentissa on tehty. Jos argumentti on täysin uusi, voit jättää tämän kohdan tyhjäksi.
            - Relevanttius: Anna jokaiselle argumentille pisteytys asteikolla 1-10 sen mukaan, kuinka hyvin muutokset vastaavat käytettyjä kommentteja. Tai jos argumentti on täysin uusi, arvioi sen relevanssi kommentteihin.
            3. Luo yleinen tiivistelmä: Luo lyhyt, korkeatasoinen yhteenveto koko analysoidusta erästä. Mitä pääkohtia huomioit kommenteissa, ja miten argumentit ovat kehittyneet niiden mukaan?

            ### Analysoitavat tiedot:
            Kommentit:
            {comments_text}

            Vanhat argumentit:
            {old_arguments_text}

            Uudet argumentit:
            {new_arguments_text}

            ### Tulostusmuoto:
            <ANALYYSI>
            ARGUMENTTI 1:
            Jäljitettävyys: [Kommentit, jotka vaikuttivat muutoksiin (jätä tyhjäksi, jos uusi argumentti)]
            Lisäykset: [Mitä lisättiin]
            Poistot: [Mitä poistettiin]
            Muutokset: [Mitkä asiat muuttuivat]
            Relevanttius: [Relevanssipisteytys 1-10]
            Perustelut: [Perustelut annetuille pisteille]
            ARGUMENTTI 2:
            Jäljitettävyys: [Kommentit, jotka vaikuttivat muutoksiin (jätä tyhjäksi, jos uusi argumentti)]
            Lisäykset: [Mitä lisättiin]
            Poistot: [Mitä poistettiin]
            Muutokset: [Mitkä asiat muuttuivat]
            Relevanttius: [Relevanssipisteytys 1-10]
            Perustelut: [Perustelut annetuille pisteille]
            </ANALYYSI>

            <YHTEENVETO>
            Yhteenveto kaikista tapahtuneista muutoksista: [Yleinen yhteenveto argumenttien muutoksista, poistoista ja lisäyksistä]
            Muutosten perustelut: [Millä tavalla muutokset on perusteltu]
            Kehityskohteet: [Parannusehdotukset seuraaviin muutoksiin]
            </YHTEENVETO>
        """

    async def process_comments(self, comments: List[str], topic: str) -> List[Argument]:
        """Process all comments in batches, focusing on argument evolution."""
        
        # Process initial batch to create 2-4 arguments
        if not self.current_arguments:
            initial_batch = comments[:self.batch_size]
            await self._process_initial_batch(initial_batch, topic)
        
        # Process remaining batches
        for i in range(self.batch_size, len(comments), self.batch_size):
            batch = comments[i:i + self.batch_size]
            await self._synthesize_batch(batch, topic)
            
        return self.current_arguments

    async def _process_initial_batch(self, comments: List[str], topic: str):
        """Generate initial set of 2-4 arguments from first batch of comments."""
        comments_text = "\n".join(f"Kommentti {i+1}: {comment}" 
                                for i, comment in enumerate(comments))
        
        prompt = self.INITIAL_TEMPLATE.format(
            topic=topic,
            comments_text=comments_text
        )
        
        response = await self.llm_provider.generate([Message(Role.USER, prompt)])
        new_arguments = self.format_arguments(response.content)
        
        # Create initial arguments
        self.current_arguments = [
            Argument(
                main_argument=arg,
                argument_history=[]
            ) for arg in new_arguments
        ]

    async def _synthesize_batch(self, new_comments: List[str], topic: str):
        """Synthesize new comments with existing arguments."""
        current_args_text = "\n".join(
            f"Argumentti {i+1}: {arg.main_argument}" 
            for i, arg in enumerate(self.current_arguments)
        )
        
        new_comments_text = "\n".join(
            f"Kommentti {i+1}: {comment}" 
            for i, comment in enumerate(new_comments)
        )
        
        prompt = self.SYNTHESIS_TEMPLATE.format(
            topic=topic,
            existing_arguments=current_args_text,
            new_comments=new_comments_text
        )
        
        response = await self.llm_provider.generate([Message(Role.USER, prompt)])
        new_argument_points = self.format_arguments(response.content)
        
        # Update arguments while preserving history
        await self.analyze_arguments(new_argument_points, new_comments, topic) # IMPLEMENT _update_arguments

    def format_arguments(self, response: str) -> List[str]:
        """Extract argument main points from LLM response."""
        pattern = r'<ARGUMENTS>(.*?)</ARGUMENTS>'
        match = re.search(pattern, response, re.DOTALL)
        if not match:
            return []

        arguments = []
        for line in match.group(1).strip().split('\n'):
            line = line.strip()
            if line.startswith('ARGUMENTTI:'):
                arguments.append(line.replace('ARGUMENTTI:', '').strip())
        
        return arguments
    
    def parse_arguments(self, analysis_text) -> List[Dict[str, str]]:
        """Parse the arguments from the analysis text."""
        parser = BatchAnalysisParser(analysis_text)
        return parser.parse_arguments()
    
    def parse_summary(self, analysis_text) -> Dict[str, str]:
        """Parse the summary from the analysis text."""
        parser = BatchAnalysisParser(analysis_text)
        return parser.parse_summary()
    
    def parse_batch_analysis(self, analysis_text) -> Dict[str, List[Dict[str, str]]]:
        """Parse the batch analysis text into arguments and summary."""
        parser = BatchAnalysisParser(analysis_text)
        return parser.parse()

    async def analyze_arguments(self, new_argument_points: List[str], new_comments: List[str], topic) -> str:
        """Use an LLM to analyze the arguments and provide feedback."""
        # populate the template with the arguments and comments
        old_arguments_text = "\n".join(
            f"Vanha argumentti {i+1}: {arg}" 
            for i, arg in enumerate(self.current_arguments)
        )

        comments_text = "\n".join(f"Kommentti {i+1}: {comment}" 
            for i, comment in enumerate(new_comments))

        new_arguments_text = "\n".join(
            f"Uusi argumentti {i+1}: {arg}" 
            for i, arg in enumerate(new_argument_points)
        )

        prompt = self.BATCH_METRIC_TEMPLATE.format(
            topic=topic,
            comments_text=comments_text,
            old_arguments_text=old_arguments_text,
            new_arguments_text=new_arguments_text
        )

        response = await self.llm_provider.generate([Message(Role.USER, prompt)])
        
        # Parse the analysis text to get the arguments and summary
        parsed_data = self.parse_batch_analysis(response.content)

        # Print the input and output for debugging
        print("Input to the model:")
        print(prompt)
        print("Output from the model:")
        print(response.content)
        return # currently debugging 

    async def get_argument_evolution_report(self) -> str:
        """Generate a readable report of how arguments evolved."""
        report = ["Argumenttien kehitysraportti", "=" * 50, ""]
        
        for i, arg in enumerate(self.current_arguments, 1):
            report.append(f"Argumentti {i}:")
            report.append(f"Nykyinen: {arg.main_argument}")
            
            if arg.argument_history:
                report.append("\nKehityshistoria:")
                for j, historical_version in enumerate(arg.argument_history, 1):
                    report.append(f"{j}. {historical_version}")
            report.extend(["", "-" * 50, ""])
            
        return "\n".join(report)

In [37]:
# config
api_key = os.getenv("OPENAI_API_KEY")
model="gpt-4o-2024-11-20"
openai_provider = OpenAIProvider(api_key, model)
data_source_path = os.path.join(parent_dir, 'data', 'sources', 'kuntavaalit2021.csv')
output_path = os.path.join(parent_dir, 'condensation', 'results', 'current_results', 'v0_results.txt')
n_comments = 20
batch_size = 10
topic = "?"

# NOT IN USE YET BUT USEFUL
# Choose a subset of comments to process
"""question_index = 10
explanation_column_name = f'q{question_index}.explanation_fi'
likert_column_name = f'q{question_index}.answer' """

# get comments
df = pd.read_csv(data_source_path)
comment_indices = df['q9.explanation_fi'].dropna()[:n_comments].index.tolist()
comments = df.loc[comment_indices, 'q9.explanation_fi'].tolist()

# process arguments
processor = ArgumentSynthesizer(openai_provider, batch_size)
arguments = await processor.process_comments(comments,topic)

# print results
""" for arg in arguments:
    print(arg.main_argument)
    print(arg.argument_history)
    print() """

Input to the model:

            ### Ohjeet:
            1. Käy läpi seuraavat kommentit ja vanhat sekä uudet argumentit aiheesta: "?".
            2. Arvioi uudet argumentit yksitellen seuraavien kohtien mukaan:
            - Jäljitettävyys: Kirjaa, mitkä kommentit vaikuttivat kunkin muuttuneen argumentin muutoksiin. Tai jos uskot argumentin olevan täysin uusi, kirjaa se.
            - Muutokset: Ilmoita, mitä lisäyksiä, poistamisia tai muutoksia argumentissa on tehty. Jos argumentti on täysin uusi, voit jättää tämän kohdan tyhjäksi.
            - Relevanttius: Anna jokaiselle argumentille pisteytys asteikolla 1-10 sen mukaan, kuinka hyvin muutokset vastaavat käytettyjä kommentteja. Tai jos argumentti on täysin uusi, arvioi sen relevanssi kommentteihin.
            3. Luo yleinen tiivistelmä: Luo lyhyt, korkeatasoinen yhteenveto koko analysoidusta erästä. Mitä pääkohtia huomioit kommenteissa, ja miten argumentit ovat kehittyneet niiden mukaan?

            ### Analysoitavat tiedot:
  

' for arg in arguments:\n    print(arg.main_argument)\n    print(arg.argument_history)\n    print() '